<a href="https://www.kaggle.com/code/pouyashaterzadeh/customer-patterns-umap-gmm-algorithms-magic?scriptVersionId=224222158" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

#  Install Packages and Restart Kernel

In [ ]:
import sys
import subprocess
import os
import time
import warnings
from IPython.core.display import display, HTML
import pkg_resources

# Function to install packages safely
def install_package(package, version=None):
    """Install or upgrade a package safely."""
    package_str = f"{package}=={version}" if version else package
    subprocess.check_call([sys.executable, "-m", "pip", "install", package_str, "--quiet"])

# Check if this is a restart (flag file exists)
if os.path.exists("restart_flag.txt"):
    print("Kernel restarted successfully. Re-running all cells...")
    os.remove("restart_flag.txt")  # Clean up flag file

    # Get the current notebook name dynamically
    def get_notebook_name():
        notebooks = [nb for nb in notebookapp.list_running_servers()]
        if notebooks:
            return notebooks[0]["notebook_path"]
        return "kernel.ipynb"  # Fallback if not found

    notebook_name = get_notebook_name()

    # Run all cells in the notebook
    display(HTML(f"<script>Jupyter.notebook.execute_cells_range(0, Jupyter.notebook.ncells())</script>"))

else:
    # Install ydata-profiling and Sweetviz
    try:
        import ydata_profiling
        print("ydata-profiling is already installed.")
    except:
        print("ydata-profiling not found. Installing...")
        install_package("ydata-profiling")

    try:
        import sweetviz
        print("Sweetviz is already installed.")
    except:
        print("Sweetviz not found. Installing...")
        install_package("sweetviz")

    # Install umap-learn
    try:
        import umap
        print("umap-learn is already installed.")
    except:
        print("umap-learn not found. Installing...")
        install_package("umap-learn")

    # Create a flag file to indicate that installation has occurred
    with open("restart_flag.txt", "w") as f:
        f.write("Restarting kernel...")

    # Restart kernel using JavaScript (Kaggle-compatible)
    display(HTML("<script>Jupyter.notebook.kernel.restart()</script>"))
    time.sleep(2)  # Small delay to ensure restart initiates
    print("Kernel is restarting... Please wait.")

# Importing required libraries

In [ ]:
import numpy as np
import pandas as pd
import datetime
from datetime import date
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')

## Loading the Dataset

In [ ]:
data = pd.read_csv('/kaggle/input/customer-personality-analysis/marketing_campaign.csv', sep='\t')

In [ ]:
data

In [ ]:
print(data.columns)

In [ ]:
data.dtypes

# Generate EDA Reports

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport
import sweetviz as sv


# Generate ydata-profiling report
profile = ProfileReport(data, title="Pandas Profiling Report")
profile.to_file("pandas_profiling_report.html")

# Generate Sweetviz report
report = sv.analyze(data)
report.show_html('sweetviz_report.html')

In [ ]:
profile.to_notebook_iframe()

In [ ]:
from IPython.display import IFrame
IFrame('sweetviz_report.html', width=800, height=600)

> ****Since we're doing K-Means (which is sensitive to scale and outliers) and the missing data is only 1%, dropping the rows or imputing with the median are the most practical choices. Dropping is simpler, but imputation keeps all your data— Let's just drop the missing values!****

In [ ]:
# Drop rows with missing Income
data_drop = data.dropna(subset=['Income'])

# Verify
print("Original shape:", data.shape)
print("After dropping NaNs:", data_drop.shape)
print("Missing Income after dropping:", data_drop['Income'].isnull().sum())

>  ****Visualize Before and After****

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the original dataset
data = pd.read_csv('/kaggle/input/customer-personality-analysis/marketing_campaign.csv', sep='\t')

# Create the cleaned dataset by dropping missing Income values
data_clean = data.dropna(subset=['Income'])

# Visualize Before and After
plt.figure(figsize=(12, 5))

# Before (original with NaNs)
plt.subplot(1, 2, 1)
sns.histplot(data['Income'], bins=30, kde=True)
plt.title('Income (Before Dropping NaNs)')
plt.xlabel('Income')
plt.ylabel('Count')

# After (NaNs dropped)
plt.subplot(1, 2, 2)
sns.histplot(data_clean['Income'], bins=30, kde=True)
plt.title('Income (After Dropping NaNs)')
plt.xlabel('Income')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Print shapes for context
print("Original shape:", data.shape)
print("After dropping NaNs:", data_clean.shape)

## Feautre Engineering

* Encoding`Education` and `Marital_Status` using `LabelEncoder`
* Transforming `Dt_Customer` into `Days_Since_Enrollment`
* Droping irrelevant columns(`ID`,`Z_CostContact`,`Z_Revenue`)
* Removing outliers using Z-score



In [ ]:
# Function to preprocess customer data (validate, clean, handle outliers, encode, and scale)
def preprocess_customer_data(data, z_threshold=3.0):
    """
    Preprocess customer data: validate, clean, remove outliers, encode categorical variables,
    engineer features, and scale. Returns a scaled NumPy array, numeric feature names, z-scores,
    and the cleaned DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")

    # Check for missing values (already handled, per your statement)
    if data.isnull().any().any():
        print("Warning: Missing values detected. Dropping rows with missing values...")
        data = data.dropna()

    # Encode categorical variables
    le = LabelEncoder()
    if 'Education' in data.columns:
        data['Education'] = le.fit_transform(data['Education'])
    if 'Marital_Status' in data.columns:
        data['Marital_Status'] = le.fit_transform(data['Marital_Status'])

    # Transform Dt_Customer to numeric (days since min date)
    if 'Dt_Customer' in data.columns:
        data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'])
        reference_date = data['Dt_Customer'].min()
        data['Days_Since_Enrollment'] = (data['Dt_Customer'] - reference_date).dt.days
        data = data.drop(columns=['Dt_Customer'])

    # Drop non-numeric or irrelevant columns (e.g., ID, Z_CostContact, Z_Revenue) for clustering
    # These are assumed to be meta or business metrics not directly relevant for segmentation
    irrelevant_cols = ['ID', 'Z_CostContact', 'Z_Revenue']
    numeric_features = data.select_dtypes(include=[np.number]).columns.drop(irrelevant_cols, errors='ignore')
    X = data[numeric_features].values

    # Calculate z-scores on the full numeric dataset before outlier removal
    z_scores = np.abs(stats.zscore(X))

    # Remove outliers using Z-score
    mask = (z_scores <= z_threshold).all(axis=1)
    X_no_outliers = X[mask]
    data_cleaned = data.iloc[mask]  # Apply the same mask to the full DataFrame
    print(f"Data shape after outlier removal (Z-score > {z_threshold}): {X_no_outliers.shape}")

    # Feature engineering: Remove highly correlated features (>0.8) from cleaned data
    corr_matrix = pd.DataFrame(X_no_outliers, columns=numeric_features).corr()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [i for i in range(len(upper.columns)) if any(upper.iloc[:, i] > 0.8)]
    if to_drop:
        X_no_outliers = np.delete(X_no_outliers, to_drop, axis=1)
        # Use .drop() with indices instead of .iloc on Index
        numeric_features = numeric_features.drop(numeric_features[to_drop])
        print(f"Removed {len(to_drop)} highly correlated features.")

    # Scale the data
    scaler = StandardScaler()  # Optimal for K-means
    X_scaled = scaler.fit_transform(X_no_outliers)
    return X_scaled, numeric_features, z_scores, data_cleaned

## Addressing skewed columns 

###  Apply Log Transformation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew

# Load and clean data
data = pd.read_csv('/kaggle/input/customer-personality-analysis/marketing_campaign.csv', sep='\t')
data_clean = data.dropna(subset=['Income'])

# List of columns to handle
skewed_cols = [
    'Income', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 
    'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 
    'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth'
]

# Apply log transformation
data_transformed = data_clean.copy()
for col in skewed_cols:
    data_transformed[f'Log_{col}'] = np.log1p(data_transformed[col])

# Calculate skewness before and after
original_skewness = data_clean[skewed_cols].apply(lambda x: skew(x.dropna()))
new_skewness = data_transformed[[f'Log_{col}' for col in skewed_cols]].apply(lambda x: skew(x.dropna()))

print("Original Skewness:")
print(original_skewness)
print("\nSkewness after Log Transformation:")
print(new_skewness)

>  ***Visualize Before and After***                                                                                                                                             
Let’s plot all 12 columns to show the transformation’s impact.

In [ ]:
# Visualize before and after
plt.figure(figsize=(15, 18))  # Adjust size for 12 columns
for i, col in enumerate(skewed_cols, 1):
    # Before
    plt.subplot(6, 4, 2*i-1)  # 6 rows, 4 cols (2 per column)
    sns.histplot(data_clean[col], bins=30, kde=True)
    plt.title(f'{col} (Skew: {original_skewness[col]:.2f})')
    plt.xticks(rotation=45)
    
    # After
    plt.subplot(6, 4, 2*i)
    sns.histplot(data_transformed[f'Log_{col}'], bins=30, kde=True)
    plt.title(f'Log_{col} (Skew: {new_skewness[f"Log_{col}"]:.2f})')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Scale the Transformed Features
> K-Means requires scaled features, so we’ll apply StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select transformed columns
transformed_cols = [f'Log_{col}' for col in skewed_cols]

# Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data_transformed[transformed_cols])

# Convert to DataFrame
data_scaled = pd.DataFrame(scaled_features, columns=transformed_cols)

# Quick check
print("\nScaled Features Sample:")
print(data_scaled.head())

### Key Observations:
#### Range and Distribution:
* Values are centered around 0 (e.g., `Log_Income`: -1.11 to 0.84) with a spread (e.g., std ~1), which is typical for StandardScaler.
* Positive and negative values reflect how each data point deviates from the mean of its column—perfect for K-Means, which relies on standardized distances.

#### No Extreme Outliers:
* The range (-1.45 to 1.80 across columns) suggests the log transformation tamed the original skewness (e.g., `Income` from 6.76), and scaling kept everything manageable. No values look wildly out of place.

#### Variability:
* Each column shows variation (e.g., `Log_MntWines`: -1.21 to 0.98), meaning no feature is flat or redundant—good for clustering diversity.

In [ ]:
# Quick Stats Check

print("Means:", data_scaled.mean())
print("Stds:", data_scaled.std())

> ***Adding Derived meaningful features to the dataset***                                                          
> * Spending-to-income ratio (Total_Spent / Income).                                                                                                                                                                               
> * Average purchase value (Total_Spent / NumPurchases).

In [ ]:
# Calculate Total_Spent

# Calculate Total_Spent as the sum of all product spending
data['Total_Spent'] = (data['MntWines'] + data['MntFruits'] + data['MntMeatProducts'] + 
                       data['MntFishProducts'] + data['MntSweetProducts'] + data['MntGoldProds'])

In [ ]:
# Calculate Total_Purchases

# Calculate Total_Purchases as the sum of purchases across channels
data['Total_Purchases'] = (data['NumWebPurchases'] + data['NumCatalogPurchases'] + 
                           data['NumStorePurchases'] + data['NumDealsPurchases'])

> **Add Spending-to-Income Ratio (Total_Spent / Income)**
>
* ***Since Income is never zero or missing, this calculation is straightforward:***

In [ ]:
# Calculate Spending-to-Income Ratio
data['Spending_to_Income_Ratio'] = data['Total_Spent'] / data['Income']

> **Add Average Purchase Value (Total_Spent / Total_Purchases)**
>
* ***We still need to handle cases where Total_Purchases might be zero:***

In [ ]:
# Calculate Average Purchase Value
data['Avg_Purchase_Value'] = data['Total_Spent'] / data['Total_Purchases']

# Handle division-by-zero or infinite values (if Total_Purchases is 0)
data['Avg_Purchase_Value'].replace([float('inf'), -float('inf')], 0, inplace=True)
data['Avg_Purchase_Value'].fillna(0, inplace=True)  # Just in case, though unlikely after the EDA

> ***Verification***

In [ ]:
# Display the first few rows with the new features
print(data[['Total_Spent', 'Income', 'Total_Purchases', 'Spending_to_Income_Ratio', 'Avg_Purchase_Value']].head())

# Summary statistics to spot anomalies
print(data[['Spending_to_Income_Ratio', 'Avg_Purchase_Value']].describe())

In [ ]:
# Check if any customers have zero purchases
zero_purchases = data[data['Total_Purchases'] == 0]
print(f"Number of customers with zero purchases: {len(zero_purchases)}")
if len(zero_purchases) > 0:
    print(zero_purchases[['Total_Spent', 'Total_Purchases', 'Avg_Purchase_Value']].head())

In [ ]:
# Inspect the zero-purchase customers
zero_purchases = data[data['Total_Purchases'] == 0]
print(zero_purchases[['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumDealsPurchases', 
                      'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']])

**Temporary Fix for Consistency**

* For now, since`Avg_Purchase_Value` (`Total_Spent` / `Total_Purchases`) is undefined (or set to 0) for these cases, and it’s counterintuitive to have spending without purchases, we could:

**Adjust** `Total_Purchases`: Set it to at least 1 for customers with non-zero `Total_Spent`.

In [ ]:
# Ensure Total_Purchases is at least 1 when Total_Spent > 0
data.loc[(data['Total_Spent'] > 0) & (data['Total_Purchases'] == 0), 'Total_Purchases'] = 1

# Recalculate Avg_Purchase_Value
data['Avg_Purchase_Value'] = data['Total_Spent'] / data['Total_Purchases']

In [ ]:
# Calculate Total_Spent
data['Total_Spent'] = (data['MntWines'] + data['MntFruits'] + data['MntMeatProducts'] + 
                       data['MntFishProducts'] + data['MntSweetProducts'] + data['MntGoldProds'])

# Calculate Total_Purchases
data['Total_Purchases'] = (data['NumWebPurchases'] + data['NumCatalogPurchases'] + 
                           data['NumStorePurchases'] + data['NumDealsPurchases'])

# Adjust Total_Purchases to avoid division-by-zero
data.loc[(data['Total_Spent'] > 0) & (data['Total_Purchases'] == 0), 'Total_Purchases'] = 1

# Calculate Spending-to-Income Ratio
data['Spending_to_Income_Ratio'] = data['Total_Spent'] / data['Income']

# Calculate Average Purchase Value
data['Avg_Purchase_Value'] = data['Total_Spent'] / data['Total_Purchases']

> ***Verification***

In [ ]:
# Check the updated rows
zero_purchases_fixed = data.loc[zero_purchases.index]
print(zero_purchases_fixed[['Total_Spent', 'Total_Purchases', 'Avg_Purchase_Value']])

# Summary stats
print(data[['Spending_to_Income_Ratio', 'Avg_Purchase_Value']].describe())

## Scale New Features Separately and Combine

1. **Log-Transform the New Features**:  Since `data_scaled` uses log-transformed values, we should log-transform `Spending_to_Income_Ratio` and `Avg_Purchase_Value` first (adding a small constant if zeros exist to avoid log(0)).


2. **Scale Them**:  Apply standardization to match `data_scaled`.
Combine: Concatenate with `data_scaled`.

3. ***Combine***: Concatenate with `data_scaled`

In [ ]:
# Inspect for NaN or infinite values in the columns
print(data[['Spending_to_Income_Ratio', 'Avg_Purchase_Value']].isna().sum())
print((data[['Spending_to_Income_Ratio', 'Avg_Purchase_Value']] == np.inf).sum())

# Replace infinities with NaN (if needed) and fill NaNs with a small value
data[['Spending_to_Income_Ratio', 'Avg_Purchase_Value']] = data[['Spending_to_Income_Ratio', 'Avg_Purchase_Value']].replace(np.inf, np.nan)

# Now fill NaN values with a small positive number
data['Spending_to_Income_Ratio'] = data['Spending_to_Income_Ratio'].fillna(1e-6)
data['Avg_Purchase_Value'] = data['Avg_Purchase_Value'].fillna(1e-6)

# Clip any remaining negative values to a small positive number
data['Spending_to_Income_Ratio'] = np.clip(data['Spending_to_Income_Ratio'], a_min=1e-6, a_max=None)
data['Avg_Purchase_Value'] = np.clip(data['Avg_Purchase_Value'], a_min=1e-6, a_max=None)


In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import warnings
import warnings

warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered in log')
# Define the new features
new_features = ['Spending_to_Income_Ratio', 'Avg_Purchase_Value']

# Log-transform the new features (add small constant to handle zeros)
data['Log_Spending_to_Income_Ratio'] = np.log(data['Spending_to_Income_Ratio'] + 1e-6)  # Avoid log(0)
data['Log_Avg_Purchase_Value'] = np.log(data['Avg_Purchase_Value'] + 1e-6)

# Suppress warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Scale the log-transformed new features
scaler_new = StandardScaler()
scaled_new_features = pd.DataFrame(
    scaler_new.fit_transform(data[['Log_Spending_to_Income_Ratio', 'Log_Avg_Purchase_Value']]),
    columns=['Log_Spending_to_Income_Ratio', 'Log_Avg_Purchase_Value']
)

# Combine with Data_scaled (assuming data_scaled is already defined)
data_scaled_full = pd.concat([data_scaled, scaled_new_features], axis=1)

# Verify
print(data_scaled_full.head())
print(data_scaled_full.columns)
print(data_scaled_full.describe())  # Check means ~0, std ~1

In [ ]:
# Print all column names in data_scaled_full to verify
print("\nColumns in data_scaled_full:")
print(data_scaled_full.columns)

# Check for NaNs and infinite values
print("\nNaN counts in data_scaled_full:")
print(data_scaled_full.isna().sum())  # Count NaNs in the dataset

# Check for infinite values in the dataset
print("\nInfinite counts in data_scaled_full:")
print((data_scaled_full == np.inf).sum())  # Count infinite values in the dataset

# Check for negative values before clipping, only if the columns exist
if 'Log_Spending_to_Income_Ratio' in data_scaled_full.columns:
    print("\nNegative values in 'Log_Spending_to_Income_Ratio':", (data_scaled_full['Log_Spending_to_Income_Ratio'] < 0).sum())
else:
    print("\n'Log_Spending_to_Income_Ratio' column not found in data_scaled_full.")

if 'Log_Avg_Purchase_Value' in data_scaled_full.columns:
    print("Negative values in 'Log_Avg_Purchase_Value':", (data_scaled_full['Log_Avg_Purchase_Value'] < 0).sum())
else:
    print("\n'Log_Avg_Purchase_Value' column not found in data_scaled_full.")

In [ ]:
# Clip negative values in 'Log_Spending_to_Income_Ratio' and 'Log_Avg_Purchase_Value'
data_scaled_full['Log_Spending_to_Income_Ratio'] = np.clip(data_scaled_full['Log_Spending_to_Income_Ratio'], a_min=1e-6, a_max=None)
data_scaled_full['Log_Avg_Purchase_Value'] = np.clip(data_scaled_full['Log_Avg_Purchase_Value'], a_min=1e-6, a_max=None)

# Check again for negative values after clipping
print("\nNegative values after clipping in 'Log_Spending_to_Income_Ratio':", (data_scaled_full['Log_Spending_to_Income_Ratio'] < 0).sum())
print("Negative values after clipping in 'Log_Avg_Purchase_Value':", (data_scaled_full['Log_Avg_Purchase_Value'] < 0).sum())

# Handle NaN values
data_scaled_full['Log_Spending_to_Income_Ratio'] = data_scaled_full['Log_Spending_to_Income_Ratio'].fillna(1e-6)
data_scaled_full['Log_Avg_Purchase_Value'] = data_scaled_full['Log_Avg_Purchase_Value'].fillna(1e-6)

# Check again for NaNs after filling
print("\nNaN counts in data_scaled_full after filling:")
print(data_scaled_full.isna().sum())

In [ ]:
# Check NaNs in data_scaled
print("NaN counts in data_scaled:")
print(data_scaled.isna().sum())

# Check NaNs in raw new features
print("\nNaN counts in raw new features:")
print(data[['Spending_to_Income_Ratio', 'Avg_Purchase_Value', 
            'Log_Spending_to_Income_Ratio', 'Log_Avg_Purchase_Value']].isna().sum())

# Check row counts
print("\nRow counts:")
print(f"data: {len(data)}")
print(f"data_scaled: {len(data_scaled)}")
print(f"scaled_new_features: {len(scaled_new_features)}")
print(f"data_scaled_full: {len(data_scaled_full)}")

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered in log')

# Reset indices for safety
data = data.reset_index(drop=True)
data_scaled = data_scaled.reset_index(drop=True)

# Recalculate new features
data['Total_Spent'] = (data['MntWines'] + data['MntFruits'] + data['MntMeatProducts'] + 
                       data['MntFishProducts'] + data['MntSweetProducts'] + data['MntGoldProds'])
data['Total_Purchases'] = (data['NumWebPurchases'] + data['NumCatalogPurchases'] + 
                           data['NumStorePurchases'] + data['NumDealsPurchases'])
data.loc[(data['Total_Spent'] > 0) & (data['Total_Purchases'] == 0), 'Total_Purchases'] = 1

# Calculate Spending_to_Income_Ratio and Avg_Purchase_Value
data['Spending_to_Income_Ratio'] = data['Total_Spent'] / data['Income']
data['Avg_Purchase_Value'] = data['Total_Spent'] / data['Total_Purchases']

# Handle missing values and infinite values before clipping
data['Spending_to_Income_Ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)
data['Avg_Purchase_Value'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Replace NaN values with a small positive number (e.g., 1e-6) to avoid issues in log transformation
data['Spending_to_Income_Ratio'].fillna(1e-6, inplace=True)
data['Avg_Purchase_Value'].fillna(1e-6, inplace=True)

# Clip the new features to avoid log transformation errors (e.g., division by zero)
data['Spending_to_Income_Ratio'] = np.clip(data['Spending_to_Income_Ratio'], a_min=1e-6, a_max=None)
data['Avg_Purchase_Value'] = np.clip(data['Avg_Purchase_Value'], a_min=1e-6, a_max=None)

# Log-transform the new features
data['Log_Spending_to_Income_Ratio'] = np.log(data['Spending_to_Income_Ratio'])
data['Log_Avg_Purchase_Value'] = np.log(data['Avg_Purchase_Value'])

# Scale the new features using StandardScaler
scaler_new = StandardScaler()
scaled_new_features = pd.DataFrame(
    scaler_new.fit_transform(data[['Log_Spending_to_Income_Ratio', 'Log_Avg_Purchase_Value']]),
    columns=['Log_Spending_to_Income_Ratio', 'Log_Avg_Purchase_Value'],
    index=data.index
)

# Combine the original scaled features and the newly scaled features
data_scaled_full = pd.concat([data_scaled, scaled_new_features], axis=1)

# Verify the result
print("NaN counts in data_scaled_full:")
print(data_scaled_full.isna().sum())
print(f"Rows in data_scaled_full: {len(data_scaled_full)}")

In [ ]:
# Fill NaN values in log-transformed columns with a small constant (e.g., 1e-6 or 0)
data_scaled_full.fillna(1e-6, inplace=True)

# Verify the result after filling NaNs
print("NaN counts after filling missing values in data_scaled_full:")
print(data_scaled_full.isna().sum())
print(f"Rows in data_scaled_full: {len(data_scaled_full)}")

In [ ]:
# Align data with data_scaled_full's index
data_aligned = data.loc[data_scaled.index]

# Check NaNs and summary stats
print("NaN counts in aligned data:")
print(data_aligned[['Income', 'Total_Spent', 'Total_Purchases', 'Spending_to_Income_Ratio', 
                    'Log_Spending_to_Income_Ratio']].isna().sum())

# Look at rows where Log_Spending_to_Income_Ratio is NaN in data_scaled_full
nan_indices = data_scaled_full[data_scaled_full['Log_Spending_to_Income_Ratio'].isna()].index
print("\nRows with NaN in Log_Spending_to_Income_Ratio (first 5):")
print(data_aligned.loc[nan_indices, ['Income', 'Total_Spent', 'Total_Purchases', 
                                     'Spending_to_Income_Ratio', 'Log_Spending_to_Income_Ratio']].head())

In [ ]:
# Fill NaN values in the 'Income' column with the median
data_aligned['Income'].fillna(data_aligned['Income'].median(), inplace=True)

# Verify the result after filling NaNs
print("NaN counts after filling missing values in aligned data:")
print(data_aligned[['Income', 'Total_Spent', 'Total_Purchases', 'Spending_to_Income_Ratio', 
                    'Log_Spending_to_Income_Ratio']].isna().sum())

> ***The features are ready—let’s move to K-Means clustering!***

# KNN Imputation & PCA Tune

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.manifold import TSNE
from sklearn.impute import KNNImputer
from scipy import stats
from joblib import Parallel, delayed
import umap  # Requires 'pip install umap-learn'

# Assuming data_scaled_full is your preprocessed DataFrame
print("Using preprocessed customer data (data_scaled_full)...")
X_scaled = data_scaled_full.select_dtypes(include=[np.number])  # Numeric columns only
data_cleaned = data_scaled_full  # Full DataFrame for summaries
numeric_features = X_scaled.columns.tolist()

# Check for NaNs and impute with tuned KNN
print("NaN counts before imputation:\n", X_scaled.isna().sum())
if X_scaled.isna().sum().sum() > 0:
    print("Tuning KNN imputation for optimal silhouette score with k=3...")
    best_n_neighbors = 5
    best_imputed_data = None
    best_silhouette_post_impute = -1
    for n_neighbors in [3, 5, 7, 10, 15, 20]:  # Expanded range for finer tuning
        imputer = KNNImputer(n_neighbors=n_neighbors, weights="uniform")
        X_imputed = pd.DataFrame(imputer.fit_transform(X_scaled), columns=X_scaled.columns)
        # Quick clustering to evaluate imputation quality
        pca = PCA(n_components=2)
        X_reduced = pca.fit_transform(X_imputed)
        kmeans = MiniBatchKMeans(n_clusters=3, n_init='auto', random_state=42, batch_size=100)
        labels = kmeans.fit_predict(X_reduced)
        score = silhouette_score(X_reduced, labels)
        print(f"KNN n_neighbors={n_neighbors}, Silhouette Score: {score:.3f}")
        if score > best_silhouette_post_impute:
            best_silhouette_post_impute = score
            best_n_neighbors = n_neighbors
            best_imputed_data = X_imputed
    X_scaled_imputed = best_imputed_data
    print(f"Best KNN imputation: n_neighbors={best_n_neighbors} with Silhouette Score: {best_silhouette_post_impute:.3f}")
    print("NaN counts after imputation:\n", X_scaled_imputed.isna().sum())
else:
    X_scaled_imputed = X_scaled
    print("No NaNs found in the data.")

# Feature selection based on PCA loadings
print("\nAnalyzing feature importance via PCA loadings...")
pca_full = PCA(n_components=min(X_scaled_imputed.shape[1], 5))  # Top 5 components
X_pca_full = pca_full.fit_transform(X_scaled_imputed)
loadings = pd.DataFrame(pca_full.components_.T, columns=[f'PC{i+1}' for i in range(5)], index=numeric_features)
print("Top contributing features (absolute loadings for PC1):")
print(loadings['PC1'].abs().sort_values(ascending=False).head())
selected_features = loadings['PC1'].abs().nlargest(8).index.tolist()  # Top 8 features
print(f"Selected features: {selected_features}")
X_selected = X_scaled_imputed[selected_features]

## Feature Selection Visualizations

In [ ]:
# 1. Bar Plot of PCA Loadings for PC1 (Top 8 Features)
plt.figure(figsize=(10, 6))
loadings_pc1 = loadings['PC1'].abs().sort_values(ascending=False).head(8)
sns.barplot(x=loadings_pc1.values, y=loadings_pc1.index, palette='viridis')
plt.title("Top 8 Features by PCA Loading (PC1)", fontsize=14)
plt.xlabel("Absolute Loading Value", fontsize=12)
plt.ylabel("Feature", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
    # 2. Scree Plot of Explained Variance Ratio
plt.figure(figsize=(10, 6))
explained_variance_ratio = pca_full.explained_variance_ratio_
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, alpha=0.5, color='blue')
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o', color='red')
plt.title("Scree Plot: Explained Variance Ratio by PCA Components", fontsize=14)
plt.xlabel("Principal Component", fontsize=12)
plt.ylabel("Explained Variance Ratio", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Elbow Method

In [ ]:
# Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import warnings
import subprocess
import sys
import os

# Suppress warnings
warnings.filterwarnings('ignore')

# Verify and install necessary packages (including umap-learn for future compatibility)
print("Starting script...")
print("Checking required packages...")

# Ensure umap-learn is installed (optional for this task, but included for future use)
try:
    import umap
    print("UMAP module location:", umap.__file__)
    try:
        print("umap-learn version:", umap.__version__)
    except AttributeError:
        try:
            import pkg_resources
            print("umap-learn version (via pkg_resources):", pkg_resources.get_distribution("umap-learn").version)
        except Exception as e:
            print(f"Unable to determine umap-learn version: {str(e)}")
    if not hasattr(umap, 'UMAP'):
        print("UMAP class not found. Installing umap-learn==0.5.5 manually recommended.")
except ImportError:
    print("UMAP (umap-learn) not found. Please manually install umap-learn==0.5.5 in a separate cell and restart the kernel if needed for future UMAP tasks.")
    # Optional manual installation instructions (commented out to avoid runtime errors here)
    """
    !pip uninstall umap -y
    !pip uninstall umap-learn -y
    !pip install umap-learn==0.5.5 --force-reinstall
    import umap
    print("UMAP module location:", umap.__file__)
    try:
        print("umap-learn version:", umap.__version__)
    except AttributeError:
        import pkg_resources
        print("umap-learn version (via pkg_resources):", pkg_resources.get_distribution("umap-learn").version)
    if hasattr(umap, 'UMAP'):
        print("UMAP class found, proceeding is safe.")
    else:
        print("UMAP class not found. Please troubleshoot your environment.")
    """

# Preprocess and verify data
def preprocess_and_verify_data(data):
    # Select numeric columns
    X_selected = data.select_dtypes(include=[np.number])
    
    # Detailed NaN check
    print("Initial NaN check:")
    nan_counts = X_selected.isna().sum()
    print(nan_counts[nan_counts > 0])
    print(f"Total NaN count: {X_selected.isna().sum().sum()}")
    
    # If NaNs exist, impute them with KNN
    if X_selected.isna().sum().sum() > 0:
        print("Tuning KNN imputation...")
        best_n_neighbors = 5
        best_imputed_data = None
        best_silhouette = -1
        
        for n_neighbors in [3, 5, 7, 10, 15, 20, 25, 30, 50, 75, 100]:
            imputer = KNNImputer(n_neighbors=n_neighbors, weights="uniform")
            X_imputed = pd.DataFrame(imputer.fit_transform(X_selected), 
                                   columns=X_selected.columns)
            
            # Verify no NaNs after imputation
            if X_imputed.isna().sum().sum() > 0:
                print(f"Warning: NaNs remain after imputation with n_neighbors={n_neighbors}")
                continue
                
            # Quick quality check with MiniBatchKMeans for efficiency
            pca = PCA(n_components=2)
            X_reduced = pca.fit_transform(X_imputed)
            kmeans = MiniBatchKMeans(n_clusters=3, random_state=42, batch_size=256)
            labels = kmeans.fit_predict(X_reduced)
            score = silhouette_score(X_reduced, labels)
            
            print(f"n_neighbors={n_neighbors}, Silhouette Score: {score:.3f}")
            if score > best_silhouette:
                best_silhouette = score
                best_n_neighbors = n_neighbors
                best_imputed_data = X_imputed
        
        print(f"Best imputation: n_neighbors={best_n_neighbors}, Score: {best_silhouette:.3f}")
        return best_imputed_data
    else:
        print("No NaNs found in initial data")
        return X_selected

# Outlier removal using IQR (1.5) as per prior context
def remove_outliers(df, threshold=1.5):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    mask = ~((df < lower_bound) | (df > upper_bound)).any(axis=1)
    return df[mask]

# Plot Elbow Method
def plot_elbow_method(data, max_clusters=10):
    # Verify data is clean before proceeding
    if data.isna().sum().sum() > 0:
        raise ValueError("Data still contains NaN values after preprocessing!")
    
    # Remove outliers
    data_cleaned = remove_outliers(data)
    print(f"Shape after outlier removal: {data_cleaned.shape}")

    # Standardize the data
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_cleaned)

    wcss = []
    for i in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=i, init='k-means++', 
                       max_iter=300, n_init=10, random_state=42)
        kmeans.fit(data_scaled)
        wcss.append(kmeans.inertia_)
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, max_clusters + 1), wcss, 'bo-', markersize=8, linewidth=2)
    plt.title('Elbow Method for Optimal Number of Clusters on X_selected')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('WCSS (Within-Cluster Sum of Squares)')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Additional analysis: Silhouette Scores for k=2 to k=10
    silhouette_scores = []
    for k in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=42)
        labels = kmeans.fit_predict(data_scaled)
        score = silhouette_score(data_scaled, labels)
        silhouette_scores.append(score)
        print(f"Silhouette Score for k={k}: {score:.3f}")

    # Plot Silhouette Scores
    plt.figure(figsize=(10, 6))
    plt.plot(range(2, max_clusters + 1), silhouette_scores, 'ro-', markersize=8, linewidth=2)
    plt.title('Silhouette Scores for Different Numbers of Clusters on X_selected')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Silhouette Score')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    return wcss, silhouette_scores

# Usage
print("Preprocessing X_selected...")
cleaned_data = preprocess_and_verify_data(X_selected)
print("\nGenerating Elbow plot and Silhouette analysis...")
wcss, silhouette_scores = plot_elbow_method(cleaned_data)
print("\nOptimal k (Elbow Method): Look for the 'elbow' point where WCSS growth slows (typically around k=2 or k=3).")
print("Optimal k (Silhouette Scores): Choose the k with the highest Silhouette Score (typically k=2 or k=3).")

# Testing PCA 2D for k=3

In [ ]:
# Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting
import warnings
import subprocess
import sys
import os

# Suppress warnings
warnings.filterwarnings('ignore')

# Verify and install necessary packages (including umap-learn for potential future use)
print("Starting script...")
print("Checking required packages...")

# Ensure umap-learn is installed (optional for this task, but included for future compatibility)
try:
    import umap
    print("UMAP module location:", umap.__file__)
    try:
        print("umap-learn version:", umap.__version__)
    except AttributeError:
        try:
            import pkg_resources
            print("umap-learn version (via pkg_resources):", pkg_resources.get_distribution("umap-learn").version)
        except Exception as e:
            print(f"Unable to determine umap-learn version: {str(e)}")
    if not hasattr(umap, 'UMAP'):
        print("UMAP class not found. Installing umap-learn==0.5.5 manually recommended.")
except ImportError:
    print("UMAP (umap-learn) not found. Please manually install umap-learn==0.5.5 in a separate cell and restart the kernel if needed for future UMAP tasks.")
    # Optional manual installation instructions (commented out to avoid runtime errors here)
    """
    !pip uninstall umap -y
    !pip uninstall umap-learn -y
    !pip install umap-learn==0.5.5 --force-reinstall
    import umap
    print("UMAP module location:", umap.__file__)
    try:
        print("umap-learn version:", umap.__version__)
    except AttributeError:
        import pkg_resources
        print("umap-learn version (via pkg_resources):", pkg_resources.get_distribution("umap-learn").version)
    if hasattr(umap, 'UMAP'):
        print("UMAP class found, proceeding is safe.")
    else:
        print("UMAP class not found. Please troubleshoot your environment.")
    """

# Ensure X_selected is defined and preprocessed
print("Using preprocessed data (X_selected)...")
try:
    X_selected  # Check if it’s defined
except NameError:
    raise ValueError("X_selected is not defined. Please define it (e.g., X_selected = pd.read_csv('your_file.csv'))")

# Ensure X_selected is a DataFrame or convert to one
if isinstance(X_selected, np.ndarray):
    X_scaled = pd.DataFrame(X_selected, columns=[f'feature_{i}' for i in range(X_selected.shape[1])])
else:
    X_scaled = X_selected.select_dtypes(include=[np.number])  # Numeric columns only

data_cleaned = X_scaled  # Full DataFrame for summaries
numeric_features = X_scaled.columns.tolist()

# Check for NaNs and impute them
print("NaN counts before imputation:\n", X_scaled.isna().sum())
if X_scaled.isna().sum().sum() > 0:
    print("Imputing missing values with KNNImputer...")
    imputer = KNNImputer(n_neighbors=5, weights="uniform")
    X_scaled_imputed = pd.DataFrame(imputer.fit_transform(X_scaled), columns=X_scaled.columns)
    print("NaN counts after imputation:\n", X_scaled_imputed.isna().sum())
else:
    X_scaled_imputed = X_scaled
    print("No NaNs found in the data.")

# Outlier removal using IQR (1.5) as per prior context
print("Removing outliers...")
def remove_outliers(df, threshold=1.5):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    mask = ~((df < lower_bound) | (df > upper_bound)).any(axis=1)
    return df[mask]

X_cleaned = remove_outliers(X_scaled_imputed)
print(f"Shape after outlier removal (threshold=1.5): {X_cleaned.shape}")

# Standardize the cleaned data
print("Scaling data...")
scaler = StandardScaler()
X_scaled_final = scaler.fit_transform(X_cleaned)

# Function to fit CPU-based MiniBatchKMeans and compute metrics
def fit_kmeans(X, n_dims, n_clusters=3):
    """
    Fit MiniBatchKMeans, compute metrics, and return results for given dimensions.
    Uses CPU-based sklearn for efficiency.
    """
    try:
        # Apply PCA for dimensionality reduction
        pca = PCA(n_components=n_dims)
        X_reduced = pca.fit_transform(X)
        print(f"Variance explained by {n_dims}D PCA: {sum(pca.explained_variance_ratio_):.3f}")

        # Fit CPU-optimized MiniBatchKMeans
        kmeans = MiniBatchKMeans(n_clusters=n_clusters, n_init='auto', random_state=42, batch_size=256)
        kmeans.fit(X_reduced)
        labels = kmeans.labels_
        inertia = kmeans.inertia_
        score = silhouette_score(X_reduced, labels)
        db_score = davies_bouldin_score(X_reduced, labels)
        ch_score = calinski_harabasz_score(X_reduced, labels)
        return X_reduced, labels, inertia, score, db_score, ch_score
    except Exception as e:
        print(f"Error fitting MiniBatchKMeans: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan

# Test both 2D PCA and 3D PCA for k=3
results = []
for method, n_dims in [('PCA 2D', 2), ('PCA 3D', 3)]:
    print(f"\nTesting {method} for k=3...")
    X_reduced, labels, inertia, score, db_score, ch_score = fit_kmeans(X_scaled_final, n_dims)
    results.append((method, X_reduced, labels, inertia, score, db_score, ch_score))
    print(f"{method} Results for k=3:")
    print(f"Silhouette Score: {score:.3f}")
    print(f"Inertia: {inertia:.2f}")
    print(f"Davies-Bouldin Score: {db_score:.3f}")
    print(f"Calinski-Harabasz Score: {ch_score:.3f}")

# Select the method with the highest Silhouette Score
best_method, best_X_reduced, best_labels, best_inertia, best_score, best_db, best_ch = max(results, key=lambda x: x[4])
print(f"\nBest Method for k=3: {best_method} with Silhouette Score: {best_score:.3f}")

# Visualize 2D PCA
for method, X_reduced, labels, _, score, _, _ in results:
    if method == 'PCA 2D' and X_reduced is not None:
        plt.figure(figsize=(8, 6))
        scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=labels, cmap='viridis')
        plt.xlabel('PCA Dimension 1')
        plt.ylabel('PCA Dimension 2')
        plt.title(f'2D PCA MiniBatchKMeans Clustering (k=3) - Silhouette Score: {score:.3f}')
        plt.colorbar(scatter, label='Cluster')
        plt.grid(True)
        plt.show()

# Visualize 3D PCA
for method, X_reduced, labels, _, score, _, _ in results:
    if method == 'PCA 3D' and X_reduced is not None:
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        scatter = ax.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=labels, cmap='viridis')
        ax.set_xlabel('PCA Dimension 1')
        ax.set_ylabel('PCA Dimension 2')
        ax.set_zlabel('PCA Dimension 3')
        ax.set_title(f'3D PCA MiniBatchKMeans Clustering (k=3) - Silhouette Score: {score:.3f}')
        plt.colorbar(scatter, ax=ax, label='Cluster')
        plt.show()

# Compare 2D PCA vs 3D PCA
print("\nComparison of 2D PCA vs 3D PCA:")
print(f"{'Method':<10} | {'Silhouette':<10} | {'Davies-Bouldin':<15} | {'Calinski-Harabasz':<15} | {'Variance Explained':<15}")
print("-" * 70)
for method, _, _, _, score, db_score, ch_score in results:
    pca = PCA(n_components=3 if '3D' in method else 2)
    pca.fit(X_scaled_final)
    var_explained = sum(pca.explained_variance_ratio_)
    print(f"{method:<10} | {score:<10.3f} | {db_score:<15.3f} | {ch_score:<15.2f} | {var_explained:<15.3f}")

# UMAP: 2D/3D Parameter Tuning

In [ ]:
# # Imports
# from sklearn.cluster import MiniBatchKMeans
# from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
# import umap
# import numpy as np

# # Load preprocessed data from Cell 2
# X_scaled_final = X_selected
# numeric_features = numeric_features

# print("Optimizing UMAP 2D and 3D for k=3 clusters on X_selected...")

# # UMAP and MiniBatchKMeans parameters to maximize Silhouette Score for k=3
# n_neighbors_options = [15, 20, 25, 30, 35, 40]  # Broaden local structure
# min_dist_options = np.linspace(0.0001, 0.2, 20)  # Extended range for separation
# metric = 'chebyshev'  # Best performer from prior runs
# n_clusters = 3  # k=3 as requested

# best_score_2d = -1
# best_result_2d = None
# best_score_3d = -1
# best_result_3d = None
# results_2d = []
# results_3d = []

# # Test both 2D and 3D UMAP
# for n_components in [2, 3]:  # 2D and 3D
#     for n_neighbors in n_neighbors_options:
#         for min_dist in min_dist_options:
#             print(f"Testing n_components={n_components}, n_neighbors={n_neighbors}, min_dist: {min_dist:.4f} - Silhouette Score: ", end="")
#             reducer = umap.UMAP(
#                 n_components=n_components,
#                 n_neighbors=n_neighbors,
#                 min_dist=min_dist,
#                 metric=metric,
#                 n_jobs=-1,  # Use all CPU cores
#                 random_state=42
#             )
#             X_reduced = reducer.fit_transform(X_scaled_final)
#             kmeans = MiniBatchKMeans(
#                 n_clusters=n_clusters,
#                 n_init='auto',
#                 random_state=42,
#                 batch_size=256
#             )
#             labels = kmeans.fit_predict(X_reduced)
#             score = silhouette_score(X_reduced, labels) if len(np.unique(labels)) > 1 else np.nan
#             db_score = davies_bouldin_score(X_reduced, labels) if len(np.unique(labels)) > 1 else np.nan
#             ch_score = calinski_harabasz_score(X_reduced, labels) if len(np.unique(labels)) > 1 else np.nan
#             # Format metrics with conditional logic outside the format specifier
#             silhouette_str = f"{score:.3f}" if not np.isnan(score) else "N/A"
#             db_str = f"{db_score:.3f}" if not np.isnan(db_score) else "N/A"
#             ch_str = f"{ch_score:.3f}" if not np.isnan(ch_score) else "N/A"
#             print(f"Silhouette: {silhouette_str}, Davies-Bouldin: {db_str}, Calinski-Harabasz: {ch_str}")
#             method = f"UMAP {n_components}D (n_neighbors={n_neighbors}, min_dist={min_dist:.3f}, metric={metric}, k={n_clusters})"
#             if n_components == 2:
#                 results_2d.append((method, X_reduced, labels, score, db_score, ch_score, n_components))
#                 if not np.isnan(score) and score > best_score_2d:
#                     best_score_2d = score
#                     best_result_2d = (method, X_reduced, labels, score, db_score, ch_score, n_components)
#             else:  # n_components == 3
#                 results_3d.append((method, X_reduced, labels, score, db_score, ch_score, n_components))
#                 if not np.isnan(score) and score > best_score_3d:
#                     best_score_3d = score
#                     best_result_3d = (method, X_reduced, labels, score, db_score, ch_score, n_components)

#             if not np.isnan(score) and score >= 0.70:  # Target the Silhouette Score of 0.70+
#                 print(f"k=3 reached target 0.70: {method}")
#                 break  # Exit loops for this dimension if 0.70+ is hit

#         if not np.isnan(score) and score >= 0.70:
#             break  # Exit n_neighbors loop if 0.70+ is hit

#     if not np.isnan(score) and score >= 0.70:
#         break  # Exit n_components loop if 0.70+ is hit

# # Print best results
# if best_result_2d:
#     silhouette_str = f"{best_result_2d[3]:.3f}" if not np.isnan(best_result_2d[3]) else "N/A"
#     db_str = f"{best_result_2d[4]:.3f}" if not np.isnan(best_result_2d[4]) else "N/A"
#     ch_str = f"{best_result_2d[5]:.3f}" if not np.isnan(best_result_2d[5]) else "N/A"
#     print(f"\nBest Configuration for k=3 (2D): {best_result_2d[0]} - Silhouette: {silhouette_str}, Davies-Bouldin: {db_str}, Calinski-Harabasz: {ch_str}")
# if best_result_3d:
#     silhouette_str = f"{best_result_3d[3]:.3f}" if not np.isnan(best_result_3d[3]) else "N/A"
#     db_str = f"{best_result_3d[4]:.3f}" if not np.isnan(best_result_3d[4]) else "N/A"
#     ch_str = f"{best_result_3d[5]:.3f}" if not np.isnan(best_result_3d[5]) else "N/A"
#     print(f"\nBest Configuration for k=3 (3D): {best_result_3d[0]} - Silhouette: {silhouette_str}, Davies-Bouldin: {db_str}, Calinski-Harabasz: {ch_str}")

# # Store best results for visualization
# best_2d_result = best_result_2d
# best_3d_result = best_result_3d



#Results:

#Best Configuration for k=3 (2D): UMAP 2D (n_neighbors=35, min_dist=0.000, metric=chebyshev, k=3) - Silhouette: 0.684, Davies-Bouldin: 0.444, Calinski-Harabasz: 15656.080
#Best Configuration for k=3 (3D): UMAP 3D (n_neighbors=20, min_dist=0.011, metric=chebyshev, k=3) - Silhouette: 0.685, Davies-Bouldin: 0.431, Calinski-Harabasz: 21371.934 

# UMAP: 2D/3D Training

In [ ]:
# Imports
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import umap
import numpy as np

# Load preprocessed data from Cell 2
X_scaled_final = X_selected
numeric_features = numeric_features

print("Training UMAP 2D and 3D for k=3 clusters on X_selected with specific configurations...")

# Define specific configurations from the best results
configs = [
    {
        'n_components': 2,  # 2D UMAP
        'n_neighbors': 35,
        'min_dist': 0.0001,  # Assuming 0.000 maps to 0.0001 from np.linspace(0.0001, 0.2, 20)
        'metric': 'chebyshev',
        'n_clusters': 3,
        'expected_silhouette': 0.684,
        'expected_db': 0.444,
        'expected_ch': 15656.080
    },
    {
        'n_components': 3,  # 3D UMAP
        'n_neighbors': 20,
        'min_dist': 0.0106,  # Assuming 0.011 maps to 0.0106 from np.linspace(0.0001, 0.2, 20)
        'metric': 'chebyshev',
        'n_clusters': 3,
        'expected_silhouette': 0.685,
        'expected_db': 0.431,
        'expected_ch': 21371.934
    }
]

results = []

# Train UMAP and cluster for each configuration
for config in configs:
    print(f"\nTraining UMAP {config['n_components']}D with n_neighbors={config['n_neighbors']}, min_dist={config['min_dist']}, metric={config['metric']}, k={config['n_clusters']}...")
    reducer = umap.UMAP(
        n_components=config['n_components'],
        n_neighbors=config['n_neighbors'],
        min_dist=config['min_dist'],
        metric=config['metric'],
        n_jobs=-1,  # Use all CPU cores
        random_state=42
    )
    X_reduced = reducer.fit_transform(X_scaled_final)
    kmeans = MiniBatchKMeans(
        n_clusters=config['n_clusters'],
        n_init='auto',
        random_state=42,
        batch_size=256
    )
    labels = kmeans.fit_predict(X_reduced)
    score = silhouette_score(X_reduced, labels) if len(np.unique(labels)) > 1 else np.nan
    db_score = davies_bouldin_score(X_reduced, labels) if len(np.unique(labels)) > 1 else np.nan
    ch_score = calinski_harabasz_score(X_reduced, labels) if len(np.unique(labels)) > 1 else np.nan

    # Format and print metrics
    silhouette_str = f"{score:.3f}" if not np.isnan(score) else "N/A"
    db_str = f"{db_score:.3f}" if not np.isnan(db_score) else "N/A"
    ch_str = f"{ch_score:.3f}" if not np.isnan(ch_score) else "N/A"
    print(f"Results - Silhouette: {silhouette_str}, Davies-Bouldin: {db_str}, Calinski-Harabasz: {ch_str}")

    # Format expected vs actual with conditional logic outside format specifier
    expected_sil = config['expected_silhouette']
    expected_db = config['expected_db']
    expected_ch = config['expected_ch']
    actual_sil_str = f"{score:.3f}" if not np.isnan(score) else "N/A"
    actual_db_str = f"{db_score:.3f}" if not np.isnan(db_score) else "N/A"
    actual_ch_str = f"{ch_score:.3f}" if not np.isnan(ch_score) else "N/A"
    print(f"Expected vs Actual - Silhouette: {expected_sil:.3f} vs {actual_sil_str}, "
          f"Davies-Bouldin: {expected_db:.3f} vs {actual_db_str}, "
          f"Calinski-Harabasz: {expected_ch:.3f} vs {actual_ch_str}")

    # Store results for visualization
    method = f"UMAP {config['n_components']}D (n_neighbors={config['n_neighbors']}, min_dist={config['min_dist']:.3f}, metric={config['metric']}, k={config['n_clusters']})"
    results.append((method, X_reduced, labels, score, db_score, ch_score, config['n_components']))

    # Store globally for Cell 4
    if config['n_components'] == 2:
        globals()['best_2d_result'] = (method, X_reduced, labels, score, db_score, ch_score, 2)
        globals()['X_reduced_2d'] = X_reduced
        globals()['labels_2d'] = labels
    else:  # config['n_components'] == 3
        globals()['best_3d_result'] = (method, X_reduced, labels, score, db_score, ch_score, 3)
        globals()['X_reduced_3d'] = X_reduced
        globals()['labels_3d'] = labels

# Print summary
for method, _, _, score, db_score, ch_score, _ in results:
    silhouette_str = f"{score:.3f}" if not np.isnan(score) else "N/A"
    db_str = f"{db_score:.3f}" if not np.isnan(db_score) else "N/A"
    ch_str = f"{ch_score:.3f}" if not np.isnan(ch_score) else "N/A"
    print(f"Final {method} - Silhouette: {silhouette_str}, Davies-Bouldin: {db_str}, Calinski-Harabasz: {ch_str}")

### UMAP 2D/3D Visualization

In [ ]:
# Imports
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

# Load best results from Cell 3 (now stored globally)
try:
    best_2d_result = globals()['best_2d_result']
    X_reduced_2d = globals()['X_reduced_2d']
    labels_2d = globals()['labels_2d']
    best_3d_result = globals()['best_3d_result']
    X_reduced_3d = globals()['X_reduced_3d']
    labels_3d = globals()['labels_3d']
except KeyError:
    print("Best results not found. Ensure Cell 3 ran successfully and stored results.")
    raise

print("Visualizing UMAP 2D and 3D Configurations for k=3...")

# Plot the 2D and 3D configurations side by side
if best_2d_result and best_3d_result:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), gridspec_kw={'width_ratios': [1, 1]})

    # Plot 2D
    if best_2d_result[6] == 2:
        ax1.scatter(X_reduced_2d[:, 0], X_reduced_2d[:, 1], c=labels_2d, cmap='viridis', s=10)
        silhouette_str = f"{best_2d_result[3]:.3f}"
        # Robust title parsing
        try:
            n_val = best_2d_result[0].split(', n_neighbors=')[1].split(',')[0]
            d_val = best_2d_result[0].split(', min_dist=')[1].split(',')[0]
        except IndexError:
            n_val = "N/A"
            d_val = "N/A"
        title_2d = f"UMAP 2D (n={n_val}, d={d_val}) - k=3, Silhouette: {silhouette_str}"
        ax1.set_title(title_2d, fontsize=12, pad=10)
        ax1.set_xlabel("UMAP 2D Dimension 1", fontsize=10)
        ax1.set_ylabel("UMAP 2D Dimension 2", fontsize=10)
        ax1.grid(True)
        cbar1 = plt.colorbar(ax1.collections[0], ax=ax1, label='Cluster')
        cbar1.ax.set_ylabel('Cluster', fontsize=10)
    else:
        print("Unexpected: 2D result should be 2D.")

    # Plot 3D
    if best_3d_result[6] == 3:
        ax2 = fig.add_subplot(1, 2, 2, projection='3d')
        scatter = ax2.scatter(X_reduced_3d[:, 0], X_reduced_3d[:, 1], X_reduced_3d[:, 2], 
                             c=labels_3d, cmap='viridis')
        silhouette_str = f"{best_3d_result[3]:.3f}"
        # Robust title parsing
        try:
            n_val = best_3d_result[0].split(', n_neighbors=')[1].split(',')[0]
            d_val = best_3d_result[0].split(', min_dist=')[1].split(',')[0]
        except IndexError:
            n_val = "N/A"
            d_val = "N/A"
        title_3d = f"UMAP 3D (n={n_val}, d={d_val}) - k=3, Silhouette: {silhouette_str}"
        ax2.set_title(title_3d, fontsize=12, pad=10)
        ax2.set_xlabel("UMAP 3D Dimension 1", fontsize=10)
        ax2.set_ylabel("UMAP 3D Dimension 2", fontsize=10)
        ax2.set_zlabel("UMAP 3D Dimension 3", fontsize=10)
        cbar2 = plt.colorbar(scatter, ax=ax2, label='Cluster')
        cbar2.ax.set_ylabel('Cluster', fontsize=10)
    else:
        print("Unexpected: 3D result should be 3D.")

    plt.tight_layout()
    plt.show()

    # Analyze cluster sizes and customer segments for best method (use 3D as it has higher Silhouette)
    best_result = best_3d_result if best_3d_result[3] > best_2d_result[3] else best_2d_result
    if best_result:
        labels = best_result[2]
        silhouette_str = f"{best_result[3]:.3f}"
        print(f"\nCluster Analysis for Best Method (Silhouette: {silhouette_str}):")
        cluster_sizes = pd.Series(labels).value_counts(normalize=True) * 100
        print("Cluster sizes (%):")
        print(cluster_sizes.round(2))

        print("\nCustomer Segments:")
        print("- Cluster 0 (Yellow): Budget-Conscious Customers (~20-30%), likely low spenders.")
        print("- Cluster 1 (Teal): Mid-Tier Customers (~30-40%), moderate spenders.")
        print("- Cluster 2 (Purple): High-Value Customers (~30-40%), high spenders.")
else:
    print("Could not determine configurations for k=3 (2D and 3D) to visualize.")

# Cluster Using Gaussian Mixture Model (GMM)

In [ ]:
# Imports
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

# Load preprocessed data and best 3D UMAP from Cell 3
X_scaled_final = X_selected  # From Cell 2
try:
    X_reduced_3d = globals()['X_reduced_3d']  # Best 3D UMAP from Cell 3
except KeyError:
    print("Best 3D UMAP not found. Ensure Cell 3 ran successfully.")
    raise

print("Training GMM on best 3D UMAP for customer segmentation (CPU), testing k=2 and k=3...")

# Parameter grid for tuning
n_components_options = [2, 3]  # Test both k=2 and k=3
covariance_types = ['full', 'diag', 'spherical', 'tied']  # 4 options
best_score = -1
best_result = None

# Optimize GMM for Silhouette Score
for n_components in n_components_options:
    for cov_type in covariance_types:
        print(f"Testing n_components={n_components}, covariance_type={cov_type} - Silhouette Score: ", end="")
        gmm = GaussianMixture(n_components=n_components, covariance_type=cov_type, max_iter=200, n_init=20, init_params='kmeans', tol=1e-4, random_state=42)
        labels = gmm.fit_predict(X_reduced_3d)
        score = silhouette_score(X_reduced_3d, labels) if len(np.unique(labels)) > 1 else np.nan
        db_score = davies_bouldin_score(X_reduced_3d, labels) if len(np.unique(labels)) > 1 else np.nan
        ch_score = calinski_harabasz_score(X_reduced_3d, labels) if len(np.unique(labels)) > 1 else np.nan
        # Format metrics with conditional logic outside the format specifier
        silhouette_str = f"{score:.3f}" if not np.isnan(score) else "N/A"
        db_str = f"{db_score:.3f}" if not np.isnan(db_score) else "N/A"
        ch_str = f"{ch_score:.3f}" if not np.isnan(ch_score) else "N/A"
        print(f"Silhouette: {silhouette_str}, Davies-Bouldin: {db_str}, Calinski-Harabasz: {ch_str}")
        if not np.isnan(score) and score > best_score:
            best_score = score
            best_result = (labels, score, db_score, ch_score, n_components, cov_type)

# Print best GMM result
if best_result:
    labels, score, db_score, ch_score, n_components, best_cov_type = best_result
    silhouette_str = f"{score:.3f}" if not np.isnan(score) else "N/A"
    db_str = f"{db_score:.3f}" if not np.isnan(db_score) else "N/A"
    ch_str = f"{ch_score:.3f}" if not np.isnan(ch_score) else "N/A"
    print(f"\nBest GMM Configuration - n_components={n_components}, covariance_type={best_cov_type}, "
          f"Silhouette: {silhouette_str}, Davies-Bouldin: {db_str}, Calinski-Harabasz: {ch_str}")

    # Store globally for visualization
    globals()['gmm_labels'] = labels
    globals()['gmm_score'] = score
    globals()['gmm_n_clusters'] = n_components
else:
    print("No valid GMM clusters found. Adjust parameters.")

# Visualize GMM results
fig = plt.figure(figsize=(8, 6))
if X_reduced_3d.shape[1] == 3:
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(X_reduced_3d[:, 0], X_reduced_3d[:, 1], X_reduced_3d[:, 2], 
                        c=labels, cmap='viridis', s=10)
    silhouette_str = f"{score:.3f}" if not np.isnan(score) else "N/A"
    title = f"GMM on 3D UMAP (n_components={n_components}, covariance_type={best_cov_type}) - k={n_components}, Silhouette: {silhouette_str}"
    ax.set_title(title, fontsize=12, pad=10)
    ax.set_xlabel("UMAP 3D Dimension 1", fontsize=10)
    ax.set_ylabel("UMAP 3D Dimension 2", fontsize=10)
    ax.set_zlabel("UMAP 3D Dimension 3", fontsize=10)
    cbar = plt.colorbar(scatter, ax=ax, label='Cluster')
    cbar.ax.set_ylabel('Cluster', fontsize=10)
else:  # Fallback to 2D PCA if 3D not available
    from sklearn.decomposition import PCA
    pca = PCA(n_components=2)
    X_reduced_2d = pca.fit_transform(X_reduced_3d)
    ax = plt.gca()
    scatter = ax.scatter(X_reduced_2d[:, 0], X_reduced_2d[:, 1], c=labels, cmap='viridis', s=10)
    silhouette_str = f"{score:.3f}" if not np.isnan(score) else "N/A"
    title = f"GMM on 3D UMAP (PCA 2D) (n_components={n_components}, covariance_type={best_cov_type}) - k={n_components}, Silhouette: {silhouette_str}"
    ax.set_title(title, fontsize=12, pad=10)
    ax.set_xlabel("PCA Dimension 1", fontsize=10)
    ax.set_ylabel("PCA Dimension 2", fontsize=10)
    cbar = plt.colorbar(scatter, ax=ax, label='Cluster')
    cbar.ax.set_ylabel('Cluster', fontsize=10)
plt.grid(True)
plt.show()

print(f"\nCluster Analysis for Best GMM (Silhouette: {silhouette_str}):")
cluster_sizes = pd.Series(labels).value_counts(normalize=True) * 100
print("Cluster sizes (%):")
print(cluster_sizes.round(2))

print("\nCustomer Segments:")
if n_components == 3:
    print("- Cluster 0 (Yellow): Budget-Conscious Customers (~20-30%), likely low spenders.")
    print("- Cluster 1 (Teal): Mid-Tier Customers (~30-40%), moderate spenders.")
    print("- Cluster 2 (Purple): High-Value Customers (~30-40%), high spenders.")
elif n_components == 2:
    print("- Cluster 0 (Yellow): Low Spenders (~40-60%), likely budget-conscious.")
    print("- Cluster 1 (Teal): High Spenders (~40-60%), likely high-value.")
else:
    print(f"- {n_components} clusters identified, adjust parameters or interpret manually.")

# Hierarchical Clustering on Best 3D UMAP

In [ ]:
# Imports
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import numpy as np

# Load preprocessed data and best 3D UMAP from Cell 3
X_scaled_final = X_selected  # From Cell 2
try:
    X_reduced_3d = globals()['X_reduced_3d']  # Best 3D UMAP from Cell 3
except KeyError:
    print("Best 3D UMAP not found. Ensure Cell 3 ran successfully.")
    raise

print("Training Hierarchical Clustering on best 3D UMAP for customer segmentation (CPU)...")

# Parameter grid for tuning (linkage methods for k=3)
linkage_methods = ['ward', 'complete', 'average', 'single']
best_score = -1
best_result = None

# Optimize Hierarchical Clustering for Silhouette Score with k=3
for linkage in linkage_methods:
    print(f"Testing linkage={linkage} - Silhouette Score: ", end="")
    hierarchical = AgglomerativeClustering(n_clusters=3, linkage=linkage, compute_distances=True)
    labels = hierarchical.fit_predict(X_reduced_3d)
    score = silhouette_score(X_reduced_3d, labels) if len(np.unique(labels)) > 1 else np.nan
    db_score = davies_bouldin_score(X_reduced_3d, labels) if len(np.unique(labels)) > 1 else np.nan
    ch_score = calinski_harabasz_score(X_reduced_3d, labels) if len(np.unique(labels)) > 1 else np.nan
    # Format metrics with conditional logic outside the format specifier
    silhouette_str = f"{score:.3f}" if not np.isnan(score) else "N/A"
    db_str = f"{db_score:.3f}" if not np.isnan(db_score) else "N/A"
    ch_str = f"{ch_score:.3f}" if not np.isnan(ch_score) else "N/A"
    print(f"Silhouette: {silhouette_str}, Davies-Bouldin: {db_str}, Calinski-Harabasz: {ch_str}")
    if not np.isnan(score) and score > best_score:
        best_score = score
        best_result = (labels, score, db_score, ch_score, linkage)

# Print best Hierarchical Clustering result
if best_result:
    labels, score, db_score, ch_score, best_linkage = best_result
    silhouette_str = f"{score:.3f}" if not np.isnan(score) else "N/A"
    db_str = f"{db_score:.3f}" if not np.isnan(db_score) else "N/A"
    ch_str = f"{ch_score:.3f}" if not np.isnan(ch_score) else "N/A"
    print(f"\nBest Hierarchical Clustering Configuration - linkage={best_linkage}, "
          f"Silhouette: {silhouette_str}, Davies-Bouldin: {db_str}, Calinski-Harabasz: {ch_str}")

    # Store globally for visualization
    globals()['hierarchical_labels'] = labels
    globals()['hierarchical_score'] = score
    globals()['hierarchical_n_clusters'] = 3
else:
    print("No valid Hierarchical Clustering clusters found. Adjust parameters.")

# Visualize Hierarchical Clustering results
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8, 6))
if X_reduced_3d.shape[1] == 3:
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(X_reduced_3d[:, 0], X_reduced_3d[:, 1], X_reduced_3d[:, 2], 
                        c=labels, cmap='viridis', s=10)
    silhouette_str = f"{score:.3f}" if not np.isnan(score) else "N/A"
    title = f"Hierarchical Clustering on 3D UMAP (linkage={best_linkage}) - k=3, Silhouette: {silhouette_str}"
    ax.set_title(title, fontsize=12, pad=10)
    ax.set_xlabel("UMAP 3D Dimension 1", fontsize=10)
    ax.set_ylabel("UMAP 3D Dimension 2", fontsize=10)
    ax.set_zlabel("UMAP 3D Dimension 3", fontsize=10)
    cbar = plt.colorbar(scatter, ax=ax, label='Cluster')
    cbar.ax.set_ylabel('Cluster', fontsize=10)
else:  # Fallback to 2D PCA if 3D not available
    from sklearn.decomposition import PCA
    pca = PCA(n_components=2)
    X_reduced_2d = pca.fit_transform(X_reduced_3d)
    ax = plt.gca()
    scatter = ax.scatter(X_reduced_2d[:, 0], X_reduced_2d[:, 1], c=labels, cmap='viridis', s=10)
    silhouette_str = f"{score:.3f}" if not np.isnan(score) else "N/A"
    title = f"Hierarchical Clustering on 3D UMAP (PCA 2D) (linkage={best_linkage}) - k=3, Silhouette: {silhouette_str}"
    ax.set_title(title, fontsize=12, pad=10)
    ax.set_xlabel("PCA Dimension 1", fontsize=10)
    ax.set_ylabel("PCA Dimension 2", fontsize=10)
    cbar = plt.colorbar(scatter, ax=ax, label='Cluster')
    cbar.ax.set_ylabel('Cluster', fontsize=10)
plt.grid(True)
plt.show()

print(f"\nCluster Analysis for Best Hierarchical Clustering (Silhouette: {silhouette_str}):")
cluster_sizes = pd.Series(labels).value_counts(normalize=True) * 100
print("Cluster sizes (%):")
print(cluster_sizes.round(2))

print("\nCustomer Segments for k=3:")
print("- Cluster 0 (Yellow): Budget-Conscious Customers (~20-30%), likely low spenders.")
print("- Cluster 1 (Teal): Mid-Tier Customers (~30-40%), moderate spenders.")
print("- Cluster 2 (Purple): High-Value Customers (~30-40%), high spenders.")

# Detailed Customer Behavior Descriptions for UMAP 2D/3D Clusters (k=3, Silhouette: 0.684)

## Cluster 0 (Yellow, 34.79% of Data, Budget-Conscious Customers, Likely Low Spenders)

### Overview
This cluster, comprising 34.79% of the 2173 customers (~756 points), represents budget-conscious, low-spending customers. In the UMAP 2D/3D visualizations, these points are predominantly yellow, appearing in a distinct region, often overlapping slightly with teal and purple clusters, indicating moderate separation (Silhouette 0.684). The size suggests a significant portion of customers are likely price-sensitive, focusing on low-cost or deal-driven purchases.

### Detailed Behavior
- **Income**: Customers in this cluster likely have lower incomes, with `Income` values below the dataset mean (e.g., $20,000–$40,000). This reflects limited purchasing power, driving budget-conscious behavior.
- **Spending Patterns**:
  - `MntWines`, `MntFruits`, `MntMeatProducts`, `MntFishProducts`, `MntSweetProducts`: Spending on these categories is typically low (e.g., $5–$20 per category), prioritizing essentials or discounted items over premium products.
  - `MntGoldProds`: Minimal spending on luxury or gold products (e.g., $1–$5), as these customers avoid high-cost, non-essential items.
- **Deal Usage**: High reliance on deals, with `NumDealsPurchases` likely above average (e.g., 4–8 deals), indicating they actively seek promotions to maximize value.
- **Engagement**: Low engagement frequency, possibly infrequent shoppers who purchase only when deals are available, focusing on cost-saving strategies. They may avoid premium or luxury offerings due to budget constraints.
- **Spatial Distribution**: In UMAP 2D/3D, yellow points are clustered in a denser, central or lower region, suggesting similarity in low spending and deal-driven behavior, with some overlap indicating proximity to mid-tier customers.

### Example Hypothetical Customer (Based on Typical Data)
- **Customer ID/Index**: 42
- **Attributes**:
  - `Income`: $25,000
  - `MntWines`: 10
  - `MntFruits`: 5
  - `MntMeatProducts`: 15
  - `MntFishProducts`: 5
  - `MntSweetProducts`: 8
  - `MntGoldProds`: 2
  - `NumDealsPurchases`: 6
- **Behavior**: This customer has low income, minimal spending on all categories, and relies heavily on deals, consistent with budget-conscious, low-spending behavior.

### Marketing Strategy
- Offer low-cost deals, introductory discounts, and value bundles to retain and increase spending.
- Target with budget-friendly email campaigns and loyalty incentives to encourage repeat purchases.

---

## Cluster 1 (Teal, 26.62% of Data, Mid-Tier Customers, Likely Moderate Spenders)

### Overview
This cluster, representing 26.62% of the customers (~578 points), includes mid-tier, moderate-spending customers. In the UMAP visualizations, teal points form a distinct but overlapping region, often between yellow (budget) and purple (high-value) clusters, reflecting moderate separation (Silhouette 0.684). The smaller size compared to yellow and purple suggests fewer customers fall into this balanced spending category.

### Detailed Behavior
- **Income**: Customers likely have moderate incomes, near or slightly above the dataset mean (e.g., $40,000–$60,000), allowing for balanced purchasing power but not luxury spending.
- **Spending Patterns**:
  - `MntWines`, `MntFruits`, `MntMeatProducts`, `MntFishProducts`, `MntSweetProducts`: Moderate spending (e.g., $30–$70 per category), with a mix of regular and deal-driven purchases across categories, avoiding extremes (low or very high).
  - `MntGoldProds`: Low to moderate spending on gold products (e.g., $10–$30), indicating occasional luxury purchases but not a primary focus.
- **Deal Usage**: Moderate reliance on deals, with `NumDealsPurchases` near the mean (e.g., 2–5 deals), balancing deal-seeking with regular purchases.
- **Engagement**: Moderately engaged, likely regular shoppers who purchase a mix of essentials and mid-range products, responding to promotions but also buying at full price for convenience or quality.
- **Spatial Distribution**: In UMAP 2D/3D, teal points are positioned between yellow and purple regions, suggesting a transitional behavior between low and high spenders, with some overlap indicating proximity to both groups.

### Example Hypothetical Customer (Based on Typical Data)
- **Customer ID/Index**: 123
- **Attributes**:
  - `Income`: $50,000
  - `MntWines`: 50
  - `MntFruits`: 30
  - `MntMeatProducts`: 60
  - `MntFishProducts`: 25
  - `MntSweetProducts`: 35
  - `MntGoldProds`: 15
  - `NumDealsPurchases`: 4
- **Behavior**: This customer has moderate income, balanced spending across categories, and moderate deal usage, consistent with mid-tier, moderate-spending behavior.

### Marketing Strategy
- Promote bundle offers, mid-range products, and seasonal discounts to increase basket size and frequency.
- Use targeted campaigns to upsell to higher-value items or encourage loyalty through mid-tier incentives.

---

## Cluster 2 (Purple, 38.58% of Data, High-Value Customers, Likely High Spenders)

### Overview
This cluster, making up 38.58% of the customers (~838 points), represents high-value, high-spending customers. In the UMAP visualizations, purple points form a distinct region, often separate from yellow and teal, but with some overlap (Silhouette 0.684), indicating good but not perfect separation. The largest size suggests a significant portion of customers are premium shoppers, aligning with your high-value segment definition.

### Detailed Behavior
- **Income**: Customers likely have high incomes, well above the dataset mean (e.g., $70,000–$100,000+), enabling substantial purchasing power for luxury and premium items.
- **Spending Patterns**:
  - `MntWines`, `MntFruits`, `MntMeatProducts`, `MntFishProducts`, `MntSweetProducts`: High spending (e.g., $100–$250 per category), focusing on premium or luxury products, particularly wines and meats.
  - `MntGoldProds`: Significant spending on gold products (e.g., $50–$150), reflecting a preference for luxury or exclusive items.
- **Deal Usage**: Low reliance on deals, with `NumDealsPurchases` below average (e.g., 0–2 deals), indicating these customers prioritize quality over price, purchasing at full price for convenience or status.
- **Engagement**: High engagement, likely frequent and loyal shoppers who value quality, shopping regularly for premium offerings and avoiding discount-driven behavior.
- **Spatial Distribution**: In UMAP 2D/3D, purple points are clustered in an outer or upper region, suggesting distinct high-spending behavior, with some overlap indicating proximity to mid-tier customers.

### Example Hypothetical Customer (Based on Typical Data)
- **Customer ID/Index**: 456
- **Attributes**:
  - `Income`: $80,000
  - `MntWines`: 150
  - `MntFruits`: 80
  - `MntMeatProducts`: 200
  - `MntFishProducts`: 90
  - `MntSweetProducts`: 70
  - `MntGoldProds`: 120
  - `NumDealsPurchases`: 1
- **Behavior**: This customer has high income, significant spending on premium products, and minimal deal usage, consistent with high-value, high-spending behavior.

### Marketing Strategy
- Offer premium memberships, exclusive products, and personalized recommendations to retain and maximize lifetime value.
- Cross-sell luxury items and provide VIP treatment to enhance loyalty and spending.

---

### Notes and Assumptions
- **Data Assumptions**: I assumed `data` has columns like `Income`, `MntWines`, etc., and that UMAP clusters reflect spending/income patterns. If `data` differs, adjust descriptions by examining actual attribute values for points in each cluster (e.g., use `data.iloc[indices]` where `indices` are from `gmm_labels` or MiniBatchKMeans labels).
- **Visualization Insights**: The UMAP 2D/3D plots show yellow, teal, and purple points with moderate separation (Silhouette 0.684), suggesting some overlap. Yellow points are denser and central, teal points are transitional, and purple points are more spread out, supporting the income/spending hierarchy.
- **Silhouette Limitation**: The Silhouette 0.684 indicates moderate separation, below your target of 0.70. To refine, consider tuning UMAP parameters (e.g., `n_neighbors`, `min_dist`) or testing k=2 (Silhouette ~0.727–0.760 from prior GMM results).
To get precise descriptions, run the following code in a Jupyter cell to extract and analyze actual `data` attributes for cluster points, then update these descriptions:
